In [6]:
# Imports
import numpy as np
import random as rd
import math
import IOHexperimenter as IOH
from IOHexperimenter import IOH_function

#setting the random seed
rd.seed()

In [13]:
class Simple_EA_15_100_multiple_runs:
    
    def __init__(self, budget, pc):
        
        self.budget = budget
        self.d = 1
                
        # Parameter setting
        self.mu = 15 #number of parents
        self.lamda = 100 #number of offspring
        self.pc = pc #recombination probability
        self.sigma = 10/6
        
    def initialize_individual(self):
        individual = np.empty((self.d,1))
        for i in range (0,self.d):
            individual[i]= rd.uniform(-5, 5)
        return individual

    def initialize_population(self):
        population = np.zeros((self.mu,self.d))
        for k in range(0,self.mu):
            population[k] = np.transpose(self.initialize_individual())
        return population

    def recombine(self,pop):
        offspring = np.zeros((self.d))
        p1 = pop[rd.randrange(0,len(pop))]
        p2 = pop[rd.randrange(0,len(pop))]
        if rd.uniform(0,1) < self.pc:
            for k in range (0,self.d):
                random_number = rd.uniform(0,1)
                if random_number < 0.5:
                    offspring[k] = p1[k]
                else:
                    offspring[k] = p2[k]
        else:
            offspring = p1
        return offspring
    
    """def global_discrete_recombine(self, pop):
        offspring = np.zeros((self.d))
        for i in range(0, self.d):
            random_parent = pop[rd.randrange(0,len(pop))]
            offspring[i] = random_parent[i]
            
        return offspring """

    def update_sigma(self, sigma, tau):
        new_sigma = sigma * math.exp(tau * np.random.normal(0,1))
        return new_sigma

    def singlestep_mutate (self,ind, sigma):
        result = 0
        mutant = np.zeros(len(ind))
        for j in range (0, len(ind)):
            result = ind[j] + sigma * np.random.normal(0,1)
             # check boundaries first
            if result < -5:
                mutant[j] = -5
            elif result > 5:
                mutant[j] = 5
             # if result is within boundaries, change j-th mutant characteristic to result
            else:
                mutant[j] = result

        return mutant
    """ def update_sigma_vec(self, sigma_vec, global_tau, local_tau):
        updated_sigma = np.zeros(self.d)
        for i in range (0,len(sigma_vec)):
            updated_sigma[i] = sigma_vec[i] * math.exp(global_tau * np.random.normal(0,1) + local_tau * np.random.normal(0,1))

        return updated_sigma

    def indstep_mutate(self, ind, sigma_vec):
        result = 0
        mutant = np.zeros(len(ind))
        for j in range (0,len(ind)):
            result = ind[j] + sigma_vec[j] * np.random.normal(0,1)
            if result < -5:
                mutant[j] = -5
            elif result > 5:
                mutant[j] = 5
                 # if result is within boundaries, change j-th mutant characteristic to result
            else:
                mutant[j] = result
 
        return mutant
    """
    def __call__(self, func):
        
        evalcount = 0
        self.d = func.number_of_variables
        eval_budget = self.budget * self.d
        sigma = self.sigma
        tau = 1 / math.sqrt(self.d)
        #global_tau = 1 / math.sqrt(2*self.d); #for individual stepsize mutation
        #local_tau = 1 / math.sqrt(2*mathsqrt(self.d)); #for individual stepsize mutation
        
        #Initialization
        xp = self.initialize_population() #parent population
        f = np.zeros((self.mu,1)) #parents fitness
        xo = np.zeros((self.lamda,self.d)) #offspring population
        fo = np.zeros((self.lamda,1)) #offspring fitness
        x_opt = None
        f_opt = np.inf
        
        for i in range (0,self.mu):
            f[i] = func(xp[i])
            evalcount = evalcount +1
        
        # Evaluation loop
        while evalcount < eval_budget and func.final_target_hit == False :
            
            #update sigma
            sigma = self.update_sigma(sigma, tau)
            
            #create new offspring generation
            for i in range(0,self.lamda):
                offspring = self.recombine(xp)
                xo[i] = self.singlestep_mutate(offspring, sigma)
                fo[i] = func(xo[i])
                evalcount = evalcount +1
                if evalcount == eval_budget:
                    break
                
            # Selection
            m = np.concatenate((xo, fo), axis=1) # Create joint matrix of individuals and their fitness values
            sorted_m = m[np.argsort(m[:, self.d])] # sort based on fitness (lowest values highest ranking)
            
            xpnew = sorted_m[0:self.mu,0:self.d] # copy best individuals and remove fitness values
            xp = xpnew # set new population for next generation
            f = sorted_m[0:self.mu,self.d] # copy fitness values of new generation

            #check if new best individual is better than previous best and copy x and f if true
            if f[0] < f_opt:
                x_opt = xp[0]
                f_opt = f[0]

        return x_opt, f_opt

In [14]:
list = []
for i in range (1,25):
    list.append(i)

In [15]:
pc = [0.65, 0.70, 0.75, 0.80, 0.85, 0.90]

In [17]:
#for elem in pc:
from IOHexperimenter import IOHexperimenter
exp = IOHexperimenter()
exp.initialize_BBOB(list,[1],[2], 5)
exp.set_logger_location("experiments/experiment3108-1038") #+ "pc" + str(elem)[-2:] , "run")
exp([Simple_EA_15_100_multiple_runs(10000,0.70)])

Running single-threaded
1 20000 7.411651319616183e-06
1 8615 3.0185594675324146e-09
1 4715 2.5401128830464175e-09
1 18315 7.7226382358605e-09
1 20000 2.415523722336865e-08
2 20000 0.3386199785313125
2 20000 0.2572277902197111
2 20000 2.3080872549055456
2 20000 2.8735163145357148
2 20000 0.6747752820713396
3 20000 7.65053735357211e-06
3 20000 0.0003109232671176141
3 6115 7.413826788157287e-09
3 20000 0.052285741323857174
3 20000 1.7419548575853792e-08
4 10115 2.423366305360481e-09
4 10215 4.4186131929814044e-09
4 20000 1.138674408617542
4 13315 9.899035599471253e-09
4 20000 8.683546297737083e-05
5 115 0.0
5 215 0.0
5 115 0.0
5 115 0.0
5 115 0.0
6 3515 3.496708731166357e-09
6 20000 0.08524760754407629
6 3615 4.6281180589189944e-09
6 20000 8.327528389576923e-07
6 5315 4.934383970056684e-09
7 20000 93.54091559010935
7 20000 92.9460091559011
7 20000 92.96068633887359
7 20000 92.94
7 20000 92.94000000426064
8 20000 7.633891820635667e-06
8 20000 6.562872762612105e-05
8 20000 2.998412774968771

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [35]:
from IOHexperimenter import IOHexperimenter
exp = IOHexperimenter()
exp.initialize_BBOB(list,[1],[2,5,10], 1)
exp.set_logger_location("experiment2808-0922", "run")

In [56]:
ind = np.array([3,0.5, 0.003, -2.2])
mutant = np.zeros(len(ind))
for j in range (0, len(ind)):
     result = ind[j] + 0.5 * np.random.normal(0,1)
     # check boundaries first
     if result < -5:
        mutant[j] = -5
     elif result > 5:
        mutant[j] = 5
     # if result is within boundaries, change j-th mutant characteristic to result
     else:
        mutant[j] = result

In [57]:
mutant

array([ 2.71276907,  0.17461278, -0.31786894, -2.77917296])

In [3]:
algorithm = Simple_EA(5000)

In [7]:
f10 = IOH_function(10,5,1, suite = 'BBOB')
f21 = IOH_function(21,10,1, suite = 'BBOB')

In [5]:
algorithm(f10)

(array([-3.60968161, -2.58373449, -4.74331929,  4.98983707, -2.86697315]),
 1442.9254060133305)

In [6]:
f10.clear_logger()

In [8]:
algorithm(f21)

(array([ 3.83575784,  0.87308653, -4.61357279,  0.67361998, -0.97069719,
         0.01989487, -4.13696428, -4.67276495,  2.44807701,  2.66020966]),
 44.4587691052737)

In [ ]:
def update_sigma_vec(self, sigma_vec):
    updated_sigma = np.zeros(self.d)
    for i in range (0,len(sigma_vec)):
        updated_sigma[i] = sigma_vec[i] * math.exp(self.global_tau * np.random.normal(0,1) + self.local_tau * np.random.normal(0,1))
    
    return updated_sigma

def indstep_mutate(self, ind, sigma_vec):
    result = 0
    mutant = np.zeros(len(ind))
    for j in range (0,len(ind)):
        result = ind[j] + sigma_vec[j] * np.random.normal(0,1)
        if result < -5:
            mutant[j] = -5
        elif result > 5:
            mutant[j] = 5
             # if result is within boundaries, change j-th mutant characteristic to result
        else:
            mutant[j] = result
 
    return mutant
"""

vector<double> indstep_mutate (vector<double> vec, vector<double>sigma) {
	vector<double> offspring(vec.size());
	for (int i = 0; i<vec.size(); i++){
		default_random_engine generator;
		normal_distribution<double> distribution(0,1);
		double result = vec[i] + sigma[i]* distribution(generator);
		if (result < -5) {offspring[i] = -5;}
		else if (result > 5) {offspring[i] = 5;}
		else {offspring[i] = result;}
	}
	return offspring;
}


"""

In [61]:
string = "0.74"

In [70]:
string[-2:]

'74'

In [74]:
?exp

Signature:      exp(algorithms)
Type:           IOHexperimenter
String form:    <IOHexperimenter.src.IOH_benchmarking.IOHexperimenter object at 0x114838310>
File:           /usr/local/Cellar/python@3.8/3.8.5/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IOHexperimenter/src/IOH_benchmarking.py
Docstring:     
An interfact to easily run a set of algorithms on multiple functions from the IOHexperimenter.
    
Init docstring:
Create an IOHexperimenter object for benchmarking a set of algorithms on multiple functions

Initialize the functions to use by calling 'initialize_PBO', 'initialize_BBOB' or 'initialize_custom'
Set up parallellization by calling 'set_parallell'
Call docstring:
Run the experiment with the given set of algorithms

Parameters
----------
algorithms:
    A list containing algorithm functions. 
    These will get as input only one argument, of type IOH_function.


In [75]:
algorithm1 = Simple_EA_15_100(10000,pc[0])

In [76]:
algorithm2 = Simple_EA_15_100(10000,pc[1])

In [78]:
?algId

Object `algId` not found.


In [80]:
from IOHexperimenter import IOH_logger
?IOH_logger

Init signature: IOH_logger(location, foldername='run', name='', info='')
Docstring:      <no docstring>
Init docstring:
Creates a logger to be used within an IOH_problem object

Parameters
----------
location:
    The directory in which to store the results
foldername:
    The name of the folder to create to store the results
name:
    The name of the algorithm intened to run on this function
info:
    Additional information on the algorithm

Returns
------
A logger from the IOHexperimenter package
File:           /usr/local/Cellar/python@3.8/3.8.5/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IOHexperimenter/src/IOH_logger.py
Type:           type
Subclasses:     


In [81]:
?exp

Signature:      exp(algorithms)
Type:           IOHexperimenter
String form:    <IOHexperimenter.src.IOH_benchmarking.IOHexperimenter object at 0x114838310>
File:           /usr/local/Cellar/python@3.8/3.8.5/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IOHexperimenter/src/IOH_benchmarking.py
Docstring:     
An interfact to easily run a set of algorithms on multiple functions from the IOHexperimenter.
    
Init docstring:
Create an IOHexperimenter object for benchmarking a set of algorithms on multiple functions

Initialize the functions to use by calling 'initialize_PBO', 'initialize_BBOB' or 'initialize_custom'
Set up parallellization by calling 'set_parallell'
Call docstring:
Run the experiment with the given set of algorithms

Parameters
----------
algorithms:
    A list containing algorithm functions. 
    These will get as input only one argument, of type IOH_function.


In [82]:
?exp.algorithms

Object `exp.algorithms` not found.


In [83]:
?exp.docstring

Object `exp.docstring` not found.


In [84]:
?exp.type

Object `exp.type` not found.


AttributeError: 'IOHexperimenter' object has no attribute 'Type'